In [1]:
import cv2
import numpy as np
import keying
import projection
import imp
import time

In [13]:
imp.reload(keying)

<module 'keying' from '/home/ozym4nd145/Coding/Project/Vision_Project/keying.py'>

In [14]:
def show_images(lst):
    while True:
        for i in lst:
            cv2.imshow(i[0],i[1])
        ret = cv2.waitKey(0)
        if ret==27:
            break
    cv2.destroyAllWindows()

In [33]:
def project_img(img, bg, mask_corners, key_param, tola=16, tolb=50, low_thresh=0.05, erode_sz=3,
                high_thresh=0.25, sz=5, space=200, sat_mul_lo=5, sat_mul_hi=7,scale_blur=20,blur_size=3):
    key_mask = keying.get_mask(
        img, key_param[0], tola, tolb, low_thresh, high_thresh, sz, space, erode_sz)
    mod_img = keying.process_img(img, key_param[1], sat_mul_lo, sat_mul_hi)

    bgra = keying.get_bgra(mod_img, key_mask)
    res = projection.project_to_mask(bgra, mask_corners, bg.shape)
    trans_img = res[:, :, :3]  # (x,y,3)
    trans_mask = np.expand_dims(
        (res[:, :, 3]).astype(np.float32) / 255, -1)  # (x,y)

#     r1 = np.multiply(trans_img,trans_mask)
    t_mask = np.tile(trans_mask, 3)

    trans_img = trans_img.astype(np.float32)
    bg = bg.astype(np.float32)

    r1 = cv2.multiply(trans_img, t_mask)
    r2 = cv2.multiply(bg, 1 - t_mask)

    result = cv2.add(r1, r2)
    result = result.astype(np.uint8)

    light_mask = np.expand_dims(
        scale_blur * cv2.blur(trans_mask * (1 - trans_mask), (blur_size, blur_size)), -1)
    # light_mask = keying.mod_mask(light_mask, 0, 1.0)
    light_mask = np.clip(light_mask, 0.0, 1.0)
    light = bg * light_mask
    diffl = result * (1 - light_mask)
    light_result = (light + diffl).astype(np.uint8)

    return light_result

In [16]:
def get_key_param(img):
    key_region = keying.get_region(img)
    ycrcb = keying.get_params_ycrcb(img,key_region)
    hls = keying.get_params_hls(img,key_region)
    return (ycrcb,hls)

## IMG Test

In [ ]:
img = cv2.imread("./image/twoperson.png")
bg = cv2.imread("./image/bg.jpg")
mask = cv2.imread("./image/mask.jpg")

key_param = get_key_param(img)
mask_corner = projection.get_corners_mask(mask)
res,m = project_img(img,bg,mask_corner,key_param)

show_images([("i",img),("bg",bg),("m",mask),("res",res)])

## Video Test

In [39]:
green_vid_path = "./video/fight_green.mp4"
# green_vid_path = "./video/suyash_oneplus.mp4"
env_vid_path = "./video/fight.avi"
env_mask_path = "./video/fight_mask.txt"
outp_vid = "./video/fight_result.mov"
outp_vid_l = "./video/fight_result_light.mov"

In [40]:
cap_green = cv2.VideoCapture(green_vid_path)
cap_vid = cv2.VideoCapture(env_vid_path)

In [41]:
ret,img = cap_green.read()
# img = np.transpose(img,[1,0,2])
ret,bg = cap_vid.read()
with open(env_mask_path,"r") as fl:
    lines = fl.read().split("\n")[:-1]
    corners = []
    for l in lines:
        p = [int(a) for a in l.split()]
        corners.append([[p[0],p[1]],[p[2],p[3]],[p[4],p[5]],[p[6],p[7]]])
    corners = np.array(corners,dtype=np.float32)

scale = 2
img = cv2.resize(img, (img.shape[1] // scale, img.shape[0] // scale))
bg = cv2.resize(bg, (bg.shape[1] // scale, bg.shape[0] // scale))
corners = corners/2        

In [42]:
key_param = get_key_param(img)

In [43]:
fourcc = cv2.VideoWriter_fourcc(*'XVID')
fps = cap_vid.get(5)
vid = cv2.VideoWriter(outp_vid,fourcc, fps, (bg.shape[1],bg.shape[0]))

In [46]:
iteration = 0
end_iter = len(corners)

In [45]:

while True:
    start_t = time.time()
    ret1,img = cap_green.read()
    ret2,bg = cap_vid.read()
    
    if (not(ret1 and ret2 and(iteration != end_iter))):
        break
        
    img = cv2.resize(img, (img.shape[1] // scale, img.shape[0] // scale))
    bg = cv2.resize(bg, (bg.shape[1] // scale, bg.shape[0] // scale))

    
    res = project_img(img,bg,corners[iteration],key_param)
    
#     vid.write(res)
    
    end_time = time.time()
    print(end_time-start_t)
    
    cv2.imshow("result",res)
    ret = cv2.waitKey(20)
    if ret==27:
        break
    
    iteration += 1
cv2.destroyAllWindows()
vid.release()
livid.release()

0.14176392555236816
0.08239388465881348
0.07628464698791504
0.10295867919921875
0.1276395320892334
0.08257937431335449
0.08591294288635254
0.07347273826599121
0.08033204078674316
0.07332706451416016
0.07250046730041504
0.0870506763458252
0.11345887184143066
0.08999037742614746
0.11895060539245605
0.0831303596496582
0.07381224632263184
0.08223986625671387
0.0767052173614502
0.09910821914672852
0.08750581741333008
0.09988975524902344
0.09697294235229492
0.13207006454467773
0.08905601501464844
0.07490396499633789
0.07547616958618164
0.0787358283996582
0.07694268226623535
0.07787680625915527
0.08949613571166992
0.08048629760742188
0.07781577110290527
0.07751941680908203
0.08117151260375977


NameError: name 'livid' is not defined

In [ ]:
def test():
    ret1,img = cap_green.read()
    ret2,bg = cap_vid.read()
    ret3,mask = cap_mask.read()
    
    if (not(ret1 and ret2 and ret3)):
        break
        
#     img = cv2.resize(img,(img.shape[1]//2,img.shape[0]//2))
#     bg = cv2.resize(bg,(bg.shape[1]//2,bg.shape[0]//2))
#     mask = cv2.resize(mask,(mask.shape[1]//2,mask.shape[0]//2))
#     img = np.transpose(img,[1,0,2])    
    
    res,m = project_img(img,bg,mask,key_param)
    r = np.expand_dims(10*cv2.blur(m*(1-m),(5,5)),-1)
    light = bg*r
    diffl = res*(1-r)
    newi = (light+diffl).astype(np.uint8)
    
    vid.write(res)
    livid.write(newi)

In [ ]:
%load_ext line_profiler

In [ ]:
%lprun -f project_img test()

In [ ]:
a = np.ones((2,3,1))

In [ ]:
a

In [ ]:
np.tile(a,3)

## Preprocessing mask

In [ ]:
import cv2
import numpy as np

In [ ]:
import projection

In [ ]:
def get_corners(vid):
    corners = []
    i = 0
    while True:
        ret,mask = vid.read()
        if( not ret):
            break
        i+=1
        try:
            corner = projection.get_corners_mask(mask)
            corners.append(corner)
        except Exception as e:
            print("Exception occurred in frame %d: "%(i-1),e)
            break
    np_corners = np.array(corners)
    return np_corners

In [ ]:
def save_corners(corners,file_path):
    # each line is of form: topl.x topl.y topr.x topr.y botr.x botr.y botl.x botl.y
    with open(file_path,"w") as fl:
        for cor in corners:
            for pt in cor:
                fl.write("%d %d "%(pt[0],pt[1]))
            fl.write("\n")
        

In [ ]:
mask_vid = cv2.VideoCapture("./video/fight_mask.avi")

In [ ]:
mask_vid = cv2.VideoCapture("./video/fight.avi")

In [ ]:
try:
    projection.get_corners_mask(frames[400])
except Exception as e:
    print("Exception occurred in frame %d: "%(3),e)

In [ ]:
"%d %d"%(corners[0][0][0],corners[0][0][1])

In [ ]:
save_corners(corners,"./video/fight_green.txt")

In [ ]:
corners[]